## Assignment 9 - Monica Zhang
https://spark.apache.org/docs/latest/ml-features#lsh-operations  
https://databricks.com/blog/2017/05/09/detecting-abuse-scale-locality-sensitive-hashing-uber-engineering.html

In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

#### Add "eagerEval.enabled" to beautify the way Spark DF is displayed

In [4]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [5]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [6]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

498.7 G  498.7 G  gs://msca-bdp-tweets/final_project



In [ ]:
df = spark.read.json(path)

23/03/04 22:32:04 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
23/03/04 22:45:02 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
df = df.filter(col("tweet_text").isNotNull() & col("user").isNotNull()& col('text').isNotNull())
df.count()

99994342

In [ ]:
df = df.withColumn('hashtags', expr("entities.hashtags.text"))

In [ ]:
df = df.withColumn("hashtags", transform(col("hashtags"), lambda x: regexp_replace(lower(x), "[^a-zA-Z0-9\s]", "")))

In [ ]:
df = df.withColumn("tweet_text_clean", lower(regexp_replace(df["tweet_text"], "[^a-zA-Z0-9\s]", "")))

In [ ]:
df.limit(5)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries,hashtags,tweet_text_clean
null,Wed May 25 03:41:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529306674057601024,1529306674057601024,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,ArtValley818_,"{null, Tue May 24...","<a href=""http://t...",RT @ArtValley818_...,1653450098896,false,Instead of sendin...,"{false, Sat Feb 0...",null,[],instead of sendin...
null,Wed May 25 03:41:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529306674481074176,1529306674481074176,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,romyreiner,"{null, Tue May 24...","<a href=""http://t...",RT @romyreiner: Y...,1653450098997,false,YOU CAN KILL CHIL...,"{false, Wed Nov 0...",null,[],you can kill chil...
null,Wed May 25 03:41:...,null,"{[], null, [], []...",null,null,0,false,low,null,1529306674565201921,1529306674565201921,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,BarackObama,"{null, Wed May 25...","<a href=""http://t...",RT @BarackObama: ...,1653450099017,false,Across the countr...,"{false, Sun Jul 2...",null,[],across the countr...
null,Wed May 25 03:41:...,"[53, 140]","{[], null, [], [{...",null,"{[53, 267], {[], ...",0,false,low,null,1529306674565107714,1529306674565107714,YvonneF85886180,1529291354186055681,1529291354186055681,1434303050387177474,1434303050387177474,false,en,null,null,0,null,null,null,null,null,0,0,,josh_chagnon @Col...,null,"<a href=""http://t...",@YvonneF85886180 ...,1653450099017,true,@YvonneF85886180 ...,"{false, Thu Jul 0...",null,[],yvonnef85886180 j...
null,Wed May 25 03:41:...,"[14, 140]","{[], null, [], [{...",null,"{[14, 284], {[], ...",0,false,low,null,1529306674573590528,1529306674573590528,LoganTillman,1528895673168691201,1528895673168691201,58316736,58316736,false,en,null,false,0,null,null,null,null,null,0,0,,null,null,"<a href=""http://t...",@LoganTillman Mat...,1653450099019,true,@LoganTillman Mat...,"{false, Sat Jan 2...",null,[],logantillman matt...


In [17]:
keywords = ("student loan", "student loans", "student debt", "student debts",\
            "loan forgiveness", "student debt forgiveness", "student loan relief",\
            "college debt", "higher education debt", "financial aid", "debt-free college", "debt-free education",\
            "free college", "student loan forgiveness",\
            "studentloan", "studentloans", "studentdebt", "studentdebts", "loanforgiveness",\
            "studentdebtforgiveness", "studentloanrelief", "cancelstudentloans", 'cancelstudentdebt'\
            'cancelstudentdebtnow','nostudentdebt', "takeactioncanceldebt", 'freecollege', 'studentloanforgiveness'\
           'federalstudentloans', 'studentloandebt', 'cancelstudentloandebt') 

In [18]:
filtered_df = df.filter(
    (col('tweet_text').rlike('|'.join(keywords))) | 
    (array_join(col('hashtags'), ' ').rlike('|'.join(keywords)))
)
filtered_df.count()

626090

In [19]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Sat, 04 March 2023 17:53:55'

In [ ]:
filtered_df.write.parquet('gs://msca-bdp-students-bucket/shared_data/monicazhang/final_filtered.parquet')

In [ ]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Sat, 04 March 2023 18:30:02'